##Task1:EDA

In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
data=pd.read_csv("/content/first inten project.csv")

In [85]:
data=data.rename(columns={'number of adults':'Number_Of_Adults',
                          'number of children':'Number_Of_Children', 'number of weekend nights':'Number_Of_Weekend_Nights',
       'number of week nights':'Number_Of_Week_Nights', 'type of meal':'Type_Of_Meal', 'car parking space':'Car_Parking_Space',
       'room type':'Room_Type', 'market segment type':'Market_Segment_Type', 'average price ':'Average_Price',
       'special requests':'Special_Requests', 'booking status':'Booking_Status'})

In [86]:
data.drop(['Booking_ID'],axis=1,inplace=True)

In [87]:
# data=data.loc[~data.duplicated()]
data=data.reset_index(drop=True)

In [88]:
for i,value in enumerate(data['Number_Of_Week_Nights']):
  if value>7:
    data.drop(labels=i,inplace=True)
# data=data.loc[~data.duplicated()]
data=data.reset_index(drop=True)
for i,value in enumerate(data['Number_Of_Weekend_Nights']):
  if value>3:
    data.drop(labels=i,inplace=True)

In [89]:
data["Booking_Status"] = data["Booking_Status"].replace("Not_Canceled", 0)
data["Booking_Status"] = data["Booking_Status"].replace("Canceled", 1)

In [90]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
data['Type_Of_Meal']=label_encoder.fit_transform(data['Type_Of_Meal'])
data['Room_Type']=label_encoder.fit_transform(data['Room_Type'])

In [91]:
for i,value in enumerate(data['Number_Of_Adults']):
  if value==0:
    data['Number_Of_Adults'][i]=np.nan

<ipython-input-91-ae88037a0335>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Number_Of_Adults'][i]=np.nan


In [49]:
data.isnull().sum()

Number_Of_Adults            0
Number_Of_Children          0
Number_Of_Weekend_Nights    0
Number_Of_Week_Nights       0
Type_Of_Meal                0
Car_Parking_Space           0
Room_Type                   0
lead time                   0
Market_Segment_Type         0
repeated                    0
P-C                         0
P-not-C                     0
Average_Price               0
Special_Requests            0
date of reservation         0
Booking_Status              0
dtype: int64

In [48]:
# mean_adults=data.mean().iloc[0]
# data=data.fillna(mean_adults)

<ipython-input-48-a71a7a115c7b>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  mean_adults=data.mean().iloc[0]


##Some edits on Task1
 *based on recommendation of the report*

 tried to do k-best feature selection but gave the same accuracy

###Splitting date of reservation feature

In [92]:
#To ease the deployment
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder()
data = pd.get_dummies(data,prefix=['Market_Type'], columns=['Market_Segment_Type'])

In [93]:
data['date of reservation']=data['date of reservation'].loc[data['date of reservation']!='2018-2-29']
data['Date']=pd.to_datetime(data['date of reservation'])
data['Year']=data['Date'].dt.year
data['Month']=data['Date'].dt.month
data['Day']=data['Date'].dt.day

In [94]:
data=data.drop(['Date','date of reservation'],axis=1)

In [ ]:
data.dtypes

Number_Of_Adults             float64
Number_Of_Children             int64
Number_Of_Weekend_Nights       int64
Number_Of_Week_Nights          int64
Type_Of_Meal                   int64
Car_Parking_Space              int64
Room_Type                      int64
lead time                      int64
repeated                       int64
P-C                            int64
P-not-C                        int64
Average_Price                float64
Special_Requests               int64
Booking_Status                 int64
Market_Type_Aviation           uint8
Market_Type_Complementary      uint8
Market_Type_Corporate          uint8
Market_Type_Offline            uint8
Market_Type_Online             uint8
Year                         float64
Month                        float64
Day                          float64
dtype: object

###Outliers for lead time
*Also edited the Average Price outlier handling method*

In [95]:
upp=data['lead time'].mean()+3*data['lead time'].std()
low=data['lead time'].mean()-3*data['lead time'].std()
new_data=data.loc[((data['lead time']<upp) &(data['lead time']>low))]
data=new_data
upp=data['Average_Price'].mean()+3*data['Average_Price'].std()
low=data['Average_Price'].mean()-3*data['Average_Price'].std()
new_data=data.loc[((data['Average_Price']<upp) &(data['Average_Price']>low))]
data=new_data

In [96]:
for x,value in enumerate(data['lead time']):
  if value<0:
    data['lead time'][x]=np.nan

In [97]:
data=data.drop('P-not-C',axis=1)
data=data.drop('P-C',axis=1)
data=data.drop('repeated',axis=1)

In [98]:
data.duplicated().sum()

9965

When i tried to ignore the dulicated rows and Didn't give them attention, it gave better accuracy

In [72]:
# data=data.drop_duplicates()

Also got this idea from my peers to fill the nulls with mean value insted of losing this rows for training <br>
Huge Thanks to them

In [99]:
data=data.fillna(data.mean())

In [100]:
data=data.reset_index(drop=True)

In [101]:
from sklearn.model_selection import train_test_split
data=pd.DataFrame(data)
X=data.drop('Booking_Status',axis=1)
y=data['Booking_Status']
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.2,random_state=7)

In [ ]:
X_test['Room_Type'][242]

0

###Standarization
*tried MinMax but gave worse results*

In [102]:
from sklearn.preprocessing import StandardScaler
s=StandardScaler()
X_train=s.fit_transform(X_train)
X_test=s.fit_transform(X_test)

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# X_train=scaler.fit_transform(X_train)
# X_test=scaler.fit_transform(X_test)

##Task2

###1-Logistic Regression

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [108]:
classifier=LogisticRegression(random_state=42)

In [109]:
classifier.fit(X_train,Y_train)

LogisticRegression(random_state=42)

In [110]:
y_predicted=classifier.predict(X_test)

####Confusion Matrix

In [111]:
conf_matrix=confusion_matrix(Y_test,y_predicted)
print(conf_matrix)

[[4260  550]
 [ 918 1330]]


1330  True  -Ve Predictions <br>
550  False -Ve Predictions <br>
918  Flase +Ve Predictions <br>
4260 True  +Ve Predictions

####Classification Report and Accuracy

In [112]:
report=classification_report(Y_test,y_predicted)
print(report)

              precision    recall  f1-score   support

           0       0.82      0.89      0.85      4810
           1       0.71      0.59      0.64      2248

    accuracy                           0.79      7058
   macro avg       0.77      0.74      0.75      7058
weighted avg       0.79      0.79      0.79      7058



Accuracy=79%

In [113]:
conf_matrix

array([[4260,  550],
       [ 918, 1330]])

In [114]:
Trues=conf_matrix[0][0]+conf_matrix[1][1]
Falses=conf_matrix[1][0]+conf_matrix[0][1]
accuracy=(Trues/(Trues+Falses))*100
print(f"{accuracy:.3f}%")

79.201%


###2-KNN

In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [115]:
classifier2=KNeighborsClassifier(n_neighbors=5)
classifier2.fit(X_train,Y_train)

KNeighborsClassifier()

In [116]:
y_predicted=classifier2.predict(X_test)

####Confusion Matrix

In [117]:
conf_matrix=confusion_matrix(Y_test,y_predicted)
print(conf_matrix)

[[4331  479]
 [ 623 1625]]


####Classification Model  and Accuracy

In [118]:
report=classification_report(Y_test,y_predicted)
print(report)

              precision    recall  f1-score   support

           0       0.87      0.90      0.89      4810
           1       0.77      0.72      0.75      2248

    accuracy                           0.84      7058
   macro avg       0.82      0.81      0.82      7058
weighted avg       0.84      0.84      0.84      7058



In [119]:
Trues=conf_matrix[0][0]+conf_matrix[1][1]
Falses=conf_matrix[1][0]+conf_matrix[0][1]
accuracy=(Trues/(Trues+Falses))*100
print(f"{accuracy:.3f}%")

84.387%


In [120]:
classifier2.score(X_train,Y_train)

0.8958945839679785

###3-SVM

In [121]:
from sklearn.svm import SVC

In [122]:
classifier3=SVC()
classifier3.fit(X_train,Y_train)

SVC()

In [123]:
y_predicted=classifier3.predict(X_test)

####Confusion Matrix

In [124]:
conf_matrix=confusion_matrix(Y_test,y_predicted)
print(conf_matrix)

[[4427  383]
 [ 740 1508]]


####Classification Report and Accuracy

In [125]:
report=classification_report(Y_test,y_predicted)
print(report)

              precision    recall  f1-score   support

           0       0.86      0.92      0.89      4810
           1       0.80      0.67      0.73      2248

    accuracy                           0.84      7058
   macro avg       0.83      0.80      0.81      7058
weighted avg       0.84      0.84      0.84      7058



In [126]:
Trues=conf_matrix[0][0]+conf_matrix[1][1]
Falses=conf_matrix[1][0]+conf_matrix[0][1]
accuracy=(Trues/(Trues+Falses))*100
print(f"{accuracy:.3f}%")

84.089%


In [127]:
accuracy=accuracy_score(Y_test,y_predicted)
print(accuracy)

0.8408897704732219


###4-Decision Trees

In [128]:
from sklearn.tree import DecisionTreeClassifier
classifier4=DecisionTreeClassifier()

In [129]:
classifier4.fit(X_train,Y_train)

DecisionTreeClassifier()

In [130]:
y_predicted4=classifier4.predict(X_test)

####Confusion Matrix

In [131]:
conf_matrix=confusion_matrix(Y_test,y_predicted4)
print(conf_matrix)

[[4329  481]
 [ 501 1747]]


####Classification Report and Accuracy

In [132]:
report=classification_report(Y_test,y_predicted4)
print(report)

              precision    recall  f1-score   support

           0       0.90      0.90      0.90      4810
           1       0.78      0.78      0.78      2248

    accuracy                           0.86      7058
   macro avg       0.84      0.84      0.84      7058
weighted avg       0.86      0.86      0.86      7058



In [133]:
Trues=conf_matrix[0][0]+conf_matrix[1][1]
Falses=conf_matrix[1][0]+conf_matrix[0][1]
accuracy=(Trues/(Trues+Falses))*100
print(f"{accuracy:.3f}%")

86.087%


In [134]:
accuracy4=accuracy_score(Y_test,y_predicted4)
print(accuracy4)

0.8608671011618022


###5-Random Forest

In [103]:
from sklearn.ensemble import RandomForestClassifier
classifier5=RandomForestClassifier()

In [104]:
classifier5.fit(X_train,Y_train)

RandomForestClassifier()

In [105]:
y_predicted5=classifier5.predict(X_test)

####Confusion Matrix

In [ ]:
conf_matrix=confusion_matrix(Y_test,y_predicted5)
print(conf_matrix)

[[2716  180]
 [ 435  765]]


####Classification Report and Accuracy

In [106]:
report=classification_report(Y_test,y_predicted5)
print(report)

              precision    recall  f1-score   support

           0       0.91      0.95      0.93      4810
           1       0.88      0.80      0.84      2248

    accuracy                           0.90      7058
   macro avg       0.90      0.87      0.88      7058
weighted avg       0.90      0.90      0.90      7058



In [107]:
accuracy5=accuracy_score(Y_test,y_predicted5)
print(accuracy5)

0.9018135449135732


Just for testing the deployment

In [ ]:
x=np.where(y_predicted5==1)[0]
x

In [ ]:
X_test

,Number_Of_Adults,Number_Of_Children,Number_Of_Weekend_Nights,Number_Of_Week_Nights,Type_Of_Meal,Car_Parking_Space,Room_Type,lead time,Average_Price,Special_Requests,Market_Type_Aviation,Market_Type_Complementary,Market_Type_Corporate,Market_Type_Offline,Market_Type_Online,Year,Month,Day
17227,3.0,0,2,2,0,0,0,82,112.96,1,0,0,0,0,1,2018.0,7.0,3.0
24489,2.0,1,0,2,0,0,0,112,121.50,2,0,0,0,0,1,2018.0,7.0,15.0
7087,1.0,1,2,5,0,0,3,153,101.35,1,0,0,0,0,1,2018.0,7.0,30.0
11385,2.0,1,0,1,0,0,0,37,143.10,1,0,0,0,0,1,2018.0,3.0,25.0
242,1.0,0,2,2,0,0,0,209,152.10,1,0,0,0,0,1,2018.0,11.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24770,2.0,0,0,2,0,0,0,145,116.55,1,0,0,0,0,1,2018.0,6.0,21.0
13340,2.0,0,2,2,3,0,0,214,80.75,1,0,0,0,0,1,2018.0,7.0,29.0
16465,2.0,0,1,1,0,0,0,14,126.00,2,0,0,0,0,1,2017.0,10.0,19.0
17013,3.0,0,2,3,0,0,3,94,146.70,0,0,0,0,0,1,2018.0,7.0,7.0


###Deployment Video link

In [ ]:
# Deployment Video Link: https://drive.google.com/file/d/1omIMEzigtNY4l5W9nrAd5CupbZtLqQRh/view?usp=sharing